In [1]:
from shared_utils import catalog_utils
import pandas as pd
import geopandas as gpd

# Quickly get unique stops in D7 on the SHS

In [2]:
shared_data = catalog_utils.get_catalog(catalog_name='shared_data_catalog')

In [4]:
stops = shared_data.ca_transit_stops.read()

In [8]:
d7_shs = stops.query('district_name.str.contains("07 - Los Angeles") & meters_to_ca_state_highway <= 50')

In [45]:
d7_shs.head(3)

,org_id,agency,stop_id,stop_name,n_routes,route_ids_served,routetypes,n_arrivals,n_hours_in_service,meters_to_ca_state_highway,base64_url,district_name,geometry
114,recJk2sQrZAYUPMID,City of Glendale,190,Foothill at Hillard,3,"3, 33, 34",3,0,0,29.8,aHR0cHM6Ly9nbGVuZGFsZWNhLmdvdi9Ib21lL1Nob3dEb2...,07 - Los Angeles / Ventura,POINT (-118.21509 34.21114)
145,recJk2sQrZAYUPMID,City of Glendale,356,San Fernando at Elk,2,"12, 6",3,0,0,43.7,aHR0cHM6Ly9nbGVuZGFsZWNhLmdvdi9Ib21lL1Nob3dEb2...,07 - Los Angeles / Ventura,POINT (-118.26894 34.14146)
172,recJk2sQrZAYUPMID,City of Glendale,193,Foothill at La Canada,3,"3, 33, 34",3,0,0,10.0,aHR0cHM6Ly9nbGVuZGFsZWNhLmdvdi9Ib21lL1Nob3dEb2...,07 - Los Angeles / Ventura,POINT (-118.20499 34.20678)


Where count is greater than 1, agencies share a GTFS feed. We count each stop once per feed to prevent double-counting

In [43]:
agency_groups = d7_shs[['base64_url', 'agency']].drop_duplicates().groupby('base64_url').agency.unique()
agency_groups = pd.DataFrame(agency_groups).reset_index().rename(columns={'agency':'agency_group'})

In [34]:
df = d7_shs[['base64_url', 'agency']].drop_duplicates().groupby('base64_url').count()

In [35]:
df = df.reset_index().rename(columns={'agency':'count'})

In [36]:
d7_shs[['base64_url', 'agency']].drop_duplicates().merge(df, on='base64_url').sort_values('count', ascending=False)

,base64_url,agency,count
13,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,City of Camarillo,7
15,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,City of Moorpark,7
17,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,City of Simi Valley,7
11,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,Ventura County Transportation Commission,7
12,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,City of Thousand Oaks,7
16,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,City of Ojai,7
14,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,Gold Coast Transit District,7
2,aHR0cDovL2d0ZnMuZ2lzLmZsaXgudGVjaC9ndGZzX2dlbm...,FlixBus,2
30,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,City of Duarte,2
1,aHR0cDovL2d0ZnMuZ2lzLmZsaXgudGVjaC9ndGZzX2dlbm...,Greyhound,2


In [46]:
gdf = d7_shs.drop_duplicates(subset=['base64_url', 'stop_id']).merge(agency_groups, on='base64_url')

In [48]:
gdf.agency_group = gdf.agency_group.astype(str)

In [50]:
gdf.to_csv('d7_shs_stops_062025.csv')

In [51]:
from shared_utils import webmap_utils
from calitp_data_analysis import geography_utils

In [70]:
to_map = gdf.to_crs(geography_utils.CA_NAD83Albers_m)
to_map.geometry = to_map.buffer(50)

Simple map to share

In [53]:
import branca

In [65]:
cmap = branca.colormap.step.RdPu_05
cmap.scale(vmin = 0, vmax = 50)

In [66]:
cmap.rgb_bytes_tuple(50)

(122, 1, 119)

In [71]:
to_map['color'] = [(122, 1, 119)] * to_map.shape[0]

In [72]:
webmap_utils.set_state_export(to_map, subfolder='adhoc', filename='d7_stops_062025_v2',
                             map_title="D7 Transit Stops within 50 meters of SHS, June 2025")

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


{'state_dict': {'name': 'null',
  'layers': [{'name': 'D7 Transit Stops within 50 meters of SHS, June 2025',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/adhocd7_stops_062025_v2.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': (34.009967460117466, -118.34760887370977),
  'zoom': 13},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJENyBUcmFuc2l0IFN0b3BzIHdpdGhpbiA1MCBtZXRlcnMgb2YgU0hTLCBKdW5lIDIwMjUiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL2FkaG9jZDdfc3RvcHNfMDYyMDI1X3YyLmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNX19XSwgImxhdF9sb24iOiBbMzQuMDA5OTY3NDYwMTE3NDY2LCAtMTE4LjM0NzYwODg3MzcwOTc3XSwgInpvb20iOiAxM30='}

# Quickly get unique stops in D4 on the SHS

In [2]:
shared_data = catalog_utils.get_catalog(catalog_name='shared_data_catalog')

In [3]:
stops = shared_data.ca_transit_stops.read()

In [4]:
d4_shs = stops.query('district_name.str.contains("04") & meters_to_ca_state_highway <= 50')

In [5]:
d4_shs.head(3)

,org_id,agency,stop_id,stop_name,n_routes,route_ids_served,routetypes,n_arrivals,n_hours_in_service,meters_to_ca_state_highway,base64_url,district_name,geometry
376,recRM3c9Zfaft4V2B,Cloverdale Transit,7750840,Commerce Blvd & Wilford Ln,1,48,3,1,1,32.4,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,04 - Bay Area / Oakland,POINT (-122.71236 38.33500)
377,recDupUxInMUgxeiz,Sonoma County,7750840,Commerce Blvd & Wilford Ln,1,48,3,1,1,32.4,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,04 - Bay Area / Oakland,POINT (-122.71236 38.33500)
400,recRM3c9Zfaft4V2B,Cloverdale Transit,7725410,Hwy 12 & Oakmont Dr,2,"30X, 34",3,2,2,12.3,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,04 - Bay Area / Oakland,POINT (-122.60462 38.44491)


Where count is greater than 1, agencies share a GTFS feed. We count each stop once per feed to prevent double-counting

In [6]:
agency_groups = d4_shs[['base64_url', 'agency']].drop_duplicates().groupby('base64_url').agency.unique()
agency_groups = pd.DataFrame(agency_groups).reset_index().rename(columns={'agency':'agency_group'})

In [7]:
df = d4_shs[['base64_url', 'agency']].drop_duplicates().groupby('base64_url').count()

In [8]:
df = df.reset_index().rename(columns={'agency':'count'})

In [9]:
d4_shs[['base64_url', 'agency']].drop_duplicates().merge(df, on='base64_url').sort_values('count', ascending=False)

,base64_url,agency,count
0,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,Cloverdale Transit,2
12,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,Solano County Transit,2
1,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,Sonoma County,2
5,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,Commute.org,2
6,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,City of Menlo Park,2
11,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,Solano Transportation Authority,2
24,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,San Francisco Bay Area Rapid Transit District,1
25,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,Napa Valley Transportation Authority,1
26,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"Golden Gate Bridge, Highway and Transportation...",1
27,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,City of Mountain View,1


In [10]:
gdf = d4_shs.drop_duplicates(subset=['base64_url', 'stop_id']).merge(agency_groups, on='base64_url')

In [11]:
gdf.agency_group = gdf.agency_group.astype(str)

In [20]:
gdf

,org_id,agency,stop_id,stop_name,n_routes,route_ids_served,routetypes,n_arrivals,n_hours_in_service,meters_to_ca_state_highway,base64_url,district_name,geometry,agency_group
0,recRM3c9Zfaft4V2B,Cloverdale Transit,7750840,Commerce Blvd & Wilford Ln,1,48,3,1,1,32.4,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,04 - Bay Area / Oakland,POINT (-122.71236 38.33500),['Cloverdale Transit' 'Sonoma County']
1,recRM3c9Zfaft4V2B,Cloverdale Transit,7725410,Hwy 12 & Oakmont Dr,2,"30X, 34",3,2,2,12.3,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,04 - Bay Area / Oakland,POINT (-122.60462 38.44491),['Cloverdale Transit' 'Sonoma County']
2,recRM3c9Zfaft4V2B,Cloverdale Transit,7726320,Hwy 12 & Pythian Rd,2,"30X, 34",3,2,2,12.2,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,04 - Bay Area / Oakland,POINT (-122.57949 38.43587),['Cloverdale Transit' 'Sonoma County']
3,recRM3c9Zfaft4V2B,Cloverdale Transit,7726910,Hwy 12 & Madrone Rd,2,"30X, 34",3,2,2,7.6,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,04 - Bay Area / Oakland,POINT (-122.49750 38.34334),['Cloverdale Transit' 'Sonoma County']
4,recRM3c9Zfaft4V2B,Cloverdale Transit,7760300,Hwy 116 & Lynch Rd,1,26,3,2,1,16.6,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,04 - Bay Area / Oakland,POINT (-122.81488 38.38859),['Cloverdale Transit' 'Sonoma County']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1703,recaUmi7EJdPiG067,City of Rio Vista,1540,Western Railway Museum,1,50,3,0,0,28.1,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,04 - Bay Area / Oakland,POINT (-121.87394 38.20436),['City of Rio Vista']
1704,recsBfXgev9ICDCY1,Presidio Trust,41492,Van Ness & Union (Drop Off),1,66,3,29,15,0.2,aHR0cHM6Ly9wcmVzaWRpb2J1cy5jb20vZ3Rmcw==,04 - Oakland,POINT (-122.42409 37.79861),['Presidio Trust']
1705,recsBfXgev9ICDCY1,Presidio Trust,41520,Van Ness & Union (Pick Up),1,66,3,29,14,0.8,aHR0cHM6Ly9wcmVzaWRpb2J1cy5jb20vZ3Rmcw==,04 - Oakland,POINT (-122.42389 37.79848),['Presidio Trust']
1706,recXIO3srSe6F77pC,Mission Bay Transportation Management Agency,2335513,Berry Street and King Street,1,6283,3,26,12,12.8,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,04 - Oakland,POINT (-122.39839 37.77176),['Mission Bay Transportation Management Agency']


In [12]:
gdf.to_csv('d4_shs_stops_062025.csv')

In [13]:
from shared_utils import webmap_utils
from calitp_data_analysis import geography_utils

In [14]:
to_map = gdf.to_crs(geography_utils.CA_NAD83Albers_m)
to_map.geometry = to_map.buffer(50)

Simple map to share

In [15]:
import branca

In [16]:
cmap = branca.colormap.step.RdPu_05
cmap.scale(vmin = 0, vmax = 50)

In [17]:
cmap.rgb_bytes_tuple(50)

(122, 1, 119)

In [18]:
to_map['color'] = [(122, 1, 119)] * to_map.shape[0]

In [19]:
webmap_utils.set_state_export(to_map, subfolder='adhoc', filename='d4_stops_062025',
                             map_title="D4 Transit Stops within 50 meters of SHS, June 2025")

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


{'state_dict': {'name': 'null',
  'layers': [{'name': 'D4 Transit Stops within 50 meters of SHS, June 2025',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/adhocd4_stops_062025.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': (37.82748514531896, -122.34843644918148),
  'zoom': 13},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJENCBUcmFuc2l0IFN0b3BzIHdpdGhpbiA1MCBtZXRlcnMgb2YgU0hTLCBKdW5lIDIwMjUiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL2FkaG9jZDRfc3RvcHNfMDYyMDI1Lmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNX19XSwgImxhdF9sb24iOiBbMzcuODI3NDg1MTQ1MzE4OTYsIC0xMjIuMzQ4NDM2NDQ5MTgxNDhdLCAiem9vbSI6IDEzfQ=='}